### This file is used for Hyperparameter Tuning of parameters used in the construction of the LSTM architecture.
#### The parameters - 
#### 1. Number of Epochs
#### 2. Batch Size
#### 3. Optimization Algorithm
#### 4. Learning Rate
#### 5. Network Weight Initialization
#### 6. Dropout Regularization
#### are tuned using GridSearchCV. The remaining parameters, namely the embedding size, input and output dimensions were chosen based on trial and error of different parameter values. 

In [1]:
# Imports for Hyperparameter-Tuning
import nltk
import pandas as pd
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, Dropout, Bidirectional
from keras.models import Sequential
from sklearn.model_selection import train_test_split
import re
import tensorflow as tf
from tensorflow import keras
import numpy as np
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.constraints import maxnorm
from tensorflow.keras.optimizers import Adam 
from matplotlib import pyplot as plt
import chardet

[nltk_data] Downloading package punkt to C:\Users\Aishwarya
[nltk_data]     Singh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
with open('../data/nlp_vader_textblob_classified_data.csv', 'rb') as f:
    enc = chardet.detect(f.read())  # or readline if the file is large
    
tweetData = pd.read_csv('../data/nlp_vader_textblob_classified_data.csv', encoding = enc['encoding'], index_col=False)
tweetData

,username,verified,followersCount,content,label,date,country,replyCount,retweetCount,likeCount,...,vader_neg,vader_neu,vader_pos,vader_comp,cleantext2,class,TextBlob_Subjectivity,TextBlob_Polarity,TextBlob_Analysis,final_class
0,HuyThanhNguyen5,False,0,@unitedstandmufc @markgoldbridge get ole again...,-1.0,2022-08-31 15:12:54,India,0,0,0,...,0.046,0.819,0.134,0.7133,g e t o l e a g a i n t o t e l l ...,-1,0.600000,0.290000,1,-1
1,guaslackjack,False,125,best goal: bruno top assists: pogba ??: cristi...,1.0,2021-09-11 15:56:21,India,0,0,0,...,0.000,0.561,0.439,0.8225,b e s t g o a l : b r u n o t o p a s ...,1,0.400000,0.533333,1,1
2,gerryrb,False,935,@lost_souls_07 @kayinfinite28 @altyred2 @utdpl...,0.0,2022-09-03 14:29:34,Argentina,1,0,1,...,0.000,0.505,0.495,0.9562,i d o n o t k n o w i s t h ...,1,0.783333,0.450000,1,0
3,rickitten_,False,9,players spurs in old trafford #mufc #muntot ht...,0.0,2022-03-12 19:14:18,India,0,0,0,...,0.000,1.000,0.000,0.0000,p l a y e r s s p u r s i n o l d t r ...,0,0.200000,0.100000,1,0
4,saboteurDS,False,278,@tancredipalmeri why on earth would #mufc pay ...,-1.0,2022-08-28 12:07:34,India,0,0,2,...,0.045,0.643,0.312,0.8072,w h y o n e a r t h w o u l d p a ...,-1,0.500000,0.500000,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97393,ChelseaEleven,False,21757,"thank you for everything, Thomas Tuchel #CFC #...",NaN,2022-09-07 09:09:44+00:00,India,0,1,4,...,0.000,0.667,0.333,0.3612,t h a n k y o u f o r e v e r y t h i n ...,1,0.000000,0.000000,0,1
97394,amber_chelsea,False,2195,@Chukwuemekaa10 thanks for the pic Carney ????...,NaN,2022-08-27 13:17:33+00:00,Zimbabwe,0,0,4,...,0.000,0.580,0.420,0.4404,t h a n k s f o r t h e p i c c a r ...,1,0.200000,0.200000,1,1
97395,Dc_Solomon2,False,143,Contacts ongoing between Chelsea and Brighton ...,NaN,2022-09-07 11:25:44+00:00,Zimbabwe,0,0,1,...,0.000,1.000,0.000,0.0000,c o n t a c t s o n g o i n g b e t w e e ...,0,0.400000,0.400000,1,0
97396,Marcus_Bryan_,False,5439,Billy Gilmour left #ChelseaFC to join Brighton...,NaN,2022-09-08 16:43:30+00:00,Argentina,1,2,24,...,0.000,0.805,0.195,0.5574,b i l l y g i l m o u r l e f t t o ...,1,0.035897,0.000000,0,1


In [3]:
labels = tweetData['final_class']
labels = tf.keras.utils.to_categorical(labels, 3)
def featureEngineering(tweet):
    # Lower case tweet
    tweetMod = tweet.lower()
    # Replace URLs with a space in the message
    tweetMod = re.sub('https?:\/\/[a-zA-Z0-9@:%._\/+~#=?&;-]*', ' ', tweetMod)
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    tweetMod = re.sub('\$[a-zA-Z0-9]*', ' ', tweetMod)
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    tweetMod = re.sub('\@[a-zA-Z0-9]*', ' ', tweetMod)
    # Replace everything not a letter or apostrophe with a space
    tweetMod = re.sub('[^a-zA-Z\']', ' ', tweetMod)
    # Remove single letter words
    tweetMod = ' '.join([w for w in tweetMod.split() if len(w) > 1])

    return tweetMod


# Process for all tweets
tweetData['modTweet'] = [featureEngineering(tweet) for tweet in tweetData['content']]

def lemmatizeTweet(tweet):
    words = [word for word in word_tokenize(tweet) if (word.isalpha()==1)]
    # Remove stop words
    stop = set(stopwords.words('english'))
    words = [word for word in words if (word not in stop)]
    # Lemmatize words (first noun, then verb)
    wnl = nltk.stem.WordNetLemmatizer()
    lemmatized = [wnl.lemmatize(wnl.lemmatize(word, 'n'), 'v') for word in words]
    return " ".join(lemmatized)

tweetData['lemmatizedText'] = tweetData["modTweet"].apply(lambda x:lemmatizeTweet(x))
tweetData

,username,verified,followersCount,content,label,date,country,replyCount,retweetCount,likeCount,...,vader_pos,vader_comp,cleantext2,class,TextBlob_Subjectivity,TextBlob_Polarity,TextBlob_Analysis,final_class,modTweet,lemmatizedText
0,HuyThanhNguyen5,False,0,@unitedstandmufc @markgoldbridge get ole again...,-1.0,2022-08-31 15:12:54,India,0,0,0,...,0.134,0.7133,g e t o l e a g a i n t o t e l l ...,-1,0.600000,0.290000,1,-1,get ole again to tell long conversation that e...,get ole tell long conversation everything whee...
1,guaslackjack,False,125,best goal: bruno top assists: pogba ??: cristi...,1.0,2021-09-11 15:56:21,India,0,0,0,...,0.439,0.8225,b e s t g o a l : b r u n o t o p a s ...,1,0.400000,0.533333,1,1,best goal bruno top assists pogba cristiano wh...,best goal bruno top assist pogba cristiano tim...
2,gerryrb,False,935,@lost_souls_07 @kayinfinite28 @altyred2 @utdpl...,0.0,2022-09-03 14:29:34,Argentina,1,0,1,...,0.495,0.9562,i d o n o t k n o w i s t h ...,1,0.783333,0.450000,1,0,souls don't know is the honest answer but he's...,soul know honest answer definitely hard nose b...
3,rickitten_,False,9,players spurs in old trafford #mufc #muntot ht...,0.0,2022-03-12 19:14:18,India,0,0,0,...,0.000,0.0000,p l a y e r s s p u r s i n o l d t r ...,0,0.200000,0.100000,1,0,players spurs in old trafford mufc muntot,player spur old trafford mufc muntot
4,saboteurDS,False,278,@tancredipalmeri why on earth would #mufc pay ...,-1.0,2022-08-28 12:07:34,India,0,0,2,...,0.312,0.8072,w h y o n e a r t h w o u l d p a ...,-1,0.500000,0.500000,1,-1,why on earth would mufc pay for osimhen ronald...,earth would mufc pay osimhen ronaldo go better...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97393,ChelseaEleven,False,21757,"thank you for everything, Thomas Tuchel #CFC #...",NaN,2022-09-07 09:09:44+00:00,India,0,1,4,...,0.333,0.3612,t h a n k y o u f o r e v e r y t h i n ...,1,0.000000,0.000000,0,1,thank you for everything thomas tuchel cfc che...,thank everything thomas tuchel cfc chelsea
97394,amber_chelsea,False,2195,@Chukwuemekaa10 thanks for the pic Carney ????...,NaN,2022-08-27 13:17:33+00:00,Zimbabwe,0,0,4,...,0.420,0.4404,t h a n k s f o r t h e p i c c a r ...,1,0.200000,0.200000,1,1,thanks for the pic carney utc ktbffh chelseafc,thank pic carney utc ktbffh chelseafc
97395,Dc_Solomon2,False,143,Contacts ongoing between Chelsea and Brighton ...,NaN,2022-09-07 11:25:44+00:00,Zimbabwe,0,0,1,...,0.000,0.0000,c o n t a c t s o n g o i n g b e t w e e ...,0,0.400000,0.400000,1,0,contacts ongoing between chelsea and brighton ...,contact ongoing chelsea brighton graham potter...
97396,Marcus_Bryan_,False,5439,Billy Gilmour left #ChelseaFC to join Brighton...,NaN,2022-09-08 16:43:30+00:00,Argentina,1,2,24,...,0.195,0.5574,b i l l y g i l m o u r l e f t t o ...,1,0.035897,0.000000,0,1,billy gilmour left chelseafc to join brighton ...,billy gilmour leave chelseafc join brighton pl...


In [4]:
# Padding of sequences based on number of unique words
tokenizer = Tokenizer(num_words=37320, split=' ')
tokenizer.fit_on_texts(tweetData['lemmatizedText'].values)
X = tokenizer.texts_to_sequences(tweetData['lemmatizedText'].values)
X = pad_sequences(X)

X_train, X_test, Y_train, Y_test = train_test_split(X, labels, test_size=0.3, random_state=42)

In [5]:
#Number of Epochs + Batch Size
def build_lstm():
    model_dropout = Sequential()
    model_dropout.add(Embedding(input_dim=37320, output_dim=128, input_length=X.shape[1]))
    model_dropout.add(Dropout(0.4))
    model_dropout.add(Bidirectional(LSTM(units=256, return_sequences=True)))
    model_dropout.add(Dropout(0.4))
    model_dropout.add(Bidirectional(LSTM(units=128, return_sequences=False)))
    model_dropout.add(Dense(3, activation='softmax'))
    model_dropout.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
    return model_dropout

In [ ]:
# Create model
model = KerasClassifier(build_fn=build_lstm)

# Define the grid search parameters
batch_size = [512, 256, 128, 64]
epochs = [20, 50, 100, 200]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=10)
grid_result = grid.fit(X_train, Y_train, verbose=1)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


In [ ]:
# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))